# Organize your machine learning experiments with ScalarStop

First organize your training, validation, and test sets with subclasses of a `DataBlob` objects.

Second, you describe the architecture of your machine learning models with subclasses of `ModelTemplate` objects.

Third, you create a `Model` subclass instance that initializes a model with a `ModelTemplate` and trains it on a `DataBlob`'s training and validation sets.

Finally, you can save the hyperparameters and training metrics from many `DataBlob`s, `ModelTemplate`s, and `Model`s into a SQLite or PostgreSQL database using the `TrainStore` client.

In [1]:
import os

import scalarstop as sp

import tensorflow as tf

### Table of Contents

#### 1. [**DataBlob**: Keeping your training dataset organized](#DataBlob:-Keeping-your-training-dataset-organized)
#### 2. [**ModelTemplate**: Parameterizing your model creation](#ModelTemplate:-Parameterizing-your-model-creation)
#### 3. [**Model**: Combine your ModelTemplate with your DataBlob](#Model:Combine-your-ModelTemplate-with-your-DataBlob)
#### 4. [**TrainStore**: Save and query your training metrics in a database](#TrainStore:-Save-and-query-your-machine-learning-metrics-in-a-database)

---

### `DataBlob`: Keeping your training dataset organized

The first step to training machine learning models with ScalarStop is to encase your dataset into a `DataBlob`.

A `DataBlob` is a set of three `tf.data.Dataset` pipelines--representing your training, validation, and test sets.

When you create a `DataBlob`, variables that affect the creation of the `tf.data.Dataset` pipeline are are stored in a nested Python dataclass named `Hyperparams`. Only store simple JSON-serializable types in the `Hyperparams` dataclass.

Creating a new `DataBlob` with `Hyperparams` looks roughly like this:

```python
from typing import List, Dict
import scalarstop as sp

class my_datablob_group_name(sp.DataBlob):
    @sp.dataclass
    class Hyperparams(sp.HyperparamsType):
        a: int
        b: str
        c: Dict[str, float]
        d = List[int]

    # ... more setup below ...
```

Then, we define three methods on our `DataBlob` subclass:
 - `set_training()`
 - `set_validation()`
 - `set_test()`

Each one of them has to create a *new* instance of a `tf.data.Dataset` pipeline with data samples and labels zipped together. Typically that looks like:

```python
# Create a tf.data.Dataset for your training samples.
samples = tf.data.Dataset.from_tensor_slices([1,2,3])

# And another tf.data.Dataset for your training labels.
labels = tf.data.Dataset.from_tensor_slices([0, 1, 0])

# And zip them together.
tf.data.Dataset.zip((samples, labels))
```

Do not apply any batching at this stage. We will do that later.

Now we'll create a `DataBlob` that contains the Fashion MNIST dataset.

In [2]:
class fashion_mnist_v1(sp.DataBlob):

    @sp.dataclass
    class Hyperparams(sp.HyperparamsType):
        num_training_samples: int
    
    def __init__(self, hyperparams):
        """
        You only need to override __init__ if you want to validate
        your hyperparameters or add arguments that are not hyperparameters.

        One example of a non-hyperparameter argument would be a
        database connection URL.
        """
        if hyperparams["num_training_samples"] > 50_000:
            raise ValueError("num_training_samples should be <= 50_000")
        super().__init__(hyperparams=hyperparams)
        (self._train_images, self._train_labels), \
            (self._test_images, self._test_labels) = \
            tf.keras.datasets.fashion_mnist.load_data()

    def set_training(self) -> tf.data.Dataset:
        """The training set."""
        samples = tf.data.Dataset.from_tensor_slices(
            self._train_images[:self.hyperparams.num_training_samples]
        )
        labels = tf.data.Dataset.from_tensor_slices(
            self._train_labels[:self.hyperparams.num_training_samples]
        )
        return tf.data.Dataset.zip((samples, labels))

    def set_validation(self) -> tf.data.Dataset:
        """
        The validation set.

        In this example, the validation set does not change with the
        hyperparameters. This allows us to compare results with
        different training sets to the same validation set.

        However, if your hyperparameters specify how to engineer
        features, then you might wnat the validation set and
        training set to rely on the same hyperparameters.
        """
        samples = tf.data.Dataset.from_tensor_slices(
            self._train_images[50_000:]
        )
        labels = tf.data.Dataset.from_tensor_slices(
            self._train_labels[50_000:]
        )
        return tf.data.Dataset.zip((samples, labels))

    def set_test(self) -> tf.data.Dataset:
        """The test set. Used to evaluate models but not train them."""
        samples_tf = tf.data.Dataset.from_tensor_slices(
            self._test_images
        )
        labels_tf = tf.data.Dataset.from_tensor_slices(
            self._test_labels
        )
        return tf.data.Dataset.zip((samples, labels))

Here we create a `DataBlob` instance with a dictionary to set our `Hyperparams`.

The `DataBlob` name is computed by hashing your `DataBlob` subclass class name and the names and values of your `Hyperparams`.

In [3]:
datablob1 = fashion_mnist_v1(hyperparams=dict(num_training_samples=10))
datablob1.name

'fashion_mnist_v1-p166sf7xz19hg8n3mj8f93m8'

The `DataBlob` group name is by default the `DataBlob` subclass name.

In [4]:
datablob1.group_name

'fashion_mnist_v1'

In [5]:
print(datablob1.hyperparams)

fashion_mnist_v1.Hyperparams(num_training_samples=10)


Now we create another `DataBlob` instance with a different value for `Hyperparams`.

Note that it has a different automatically-generated `name`, but it'll have the same `group_name`.

In [6]:
datablob2 = fashion_mnist_v1(hyperparams=dict(num_training_samples=50))
datablob2.name, datablob2.group_name

('fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza', 'fashion_mnist_v1')

---
### `ModelTemplate`: Parameterizing your model creation

The `ModelTemplate` is the same concept as the `DataBlob`, but instead of three `tf.data.Dataset` s, the `ModelTemplate` creates a machine learning framework model object.

Here is an example of creating a Keras model. Building and compiling the model is parameterized by values in the `Hyperparams` dataclass.

In [7]:
class small_dense_10_way_classifier_v1(sp.ModelTemplate):

    @sp.dataclass
    class Hyperparams(sp.HyperparamsType):
        hidden_units: int
        optimizer: str = "adam"

    def new_model(self):
        model = tf.keras.Sequential(
            layers=[
                tf.keras.layers.Flatten(input_shape=(28, 28)),
                tf.keras.layers.Dense(
                    units=self.hyperparams.hidden_units,
                    activation="relu",
                ),
                tf.keras.layers.Dense(units=10)
            ],
            name=self.name,
        )
        model.compile(
            optimizer=self.hyperparams.optimizer,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"],
        )
        return model

Once again, the `ModelTemplate` has a unique name generated by hashing your subclass and the `Hyperparams`.

In [8]:
model_template = small_dense_10_way_classifier_v1(hyperparams=dict(hidden_units=3))
model_template.name

'small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs'

---
### `Model`:Combine your `ModelTemplate` with your `DataBlob`

`DataBlob`s and `ModelTemplate`s are not very useful until you bring them together with a `Model`.

A `Model` is an object created by pairing together a `ModelTemplate` instance and a `DataBlob` instance, for the purpose of training the machine learning model created by the `ModelTemplate` on the `DataBlob`'s training and validation sets.

Make sure to batch your `DataBlob` before using it.

In [9]:
datablob = datablob2.batch(2)

model = sp.KerasModel.from_model_template(
    datablob=datablob,
    model_template=model_template,
)

Once again, the `Model` has a unique name. But this time it is just a concatenation of the `DataBlob` and `ModelTemplate` names.

In [10]:
model.name

'mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza'

In [11]:
model.fit(final_epoch=2, verbose=1)

Epoch 1/2
25/25 [==============================] - 2s 98ms/step - loss: 9.4557 - accuracy: 0.1000 - val_loss: 2.3924 - val_accuracy: 0.1043
Epoch 2/2
25/25 [==============================] - 2s 89ms/step - loss: 2.3005 - accuracy: 0.1600 - val_loss: 2.3722 - val_accuracy: 0.1039


{'loss': [9.455658912658691, 2.300518035888672],
 'accuracy': [0.10000000149011612, 0.1599999964237213],
 'val_loss': [2.392390012741089, 2.37215256690979],
 'val_accuracy': [0.10429999977350235, 0.1039000004529953]}

In ScalarStop, training a machine learning model is an idempotent operation. Instead of saying, "Train for $n$ **more** epochs," we say, "Train until the model has been trained for $n$ epochs **total**."

If we call `model.fit()` again with `final_epoch()` still at 2, we get the same metrics but no training happened.

In [12]:
model.fit(final_epoch=2, verbose=1)

{'loss': [9.455658912658691, 2.300518035888672],
 'accuracy': [0.10000000149011612, 0.1599999964237213],
 'val_loss': [2.392390012741089, 2.37215256690979],
 'val_accuracy': [0.10429999977350235, 0.1039000004529953]}

Training ScalarStop `Model`s are idempotent because they keep track of how many epochs they have been trained for and the generated training metrics (e.g. loss, accuracy, etc.). This information is saved to the filesystem if you call `model.save()` and is loaded back from disk if you create a new `Model` object with `Model.from_filesystem_or_model_template()`.

In [13]:
os.makedirs("models_directory", exist_ok=True)

model.save("models_directory")

os.listdir("models_directory")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models_directory/mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza/assets


['mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza']

In [14]:
model2 = sp.KerasModel.from_filesystem(
    datablob=datablob,
    model_template=model_template,
    models_directory="models_directory",
)
print(model2.name)
model2.history

mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza


{'accuracy': [0.10000000149011612, 0.1599999964237213],
 'loss': [9.455658912658691, 2.300518035888672],
 'val_accuracy': [0.10429999977350235, 0.1039000004529953],
 'val_loss': [2.392390012741089, 2.37215256690979]}

If you provide `models_directory` as an argument to `fit()`, ScalarStop will save the model to the filesystem after every epoch.

In [15]:
_ = model2.fit(final_epoch=5, verbose=1, models_directory="models_directory")

Epoch 3/5
25/25 [==============================] - 3s 107ms/step - loss: 2.2995 - accuracy: 0.1600 - val_loss: 2.3721 - val_accuracy: 0.1039
Epoch 4/5
25/25 [==============================] - 3s 101ms/step - loss: 2.2973 - accuracy: 0.1600 - val_loss: 2.3722 - val_accuracy: 0.1039
Epoch 5/5
25/25 [==============================] - 3s 100ms/step - loss: 2.2960 - accuracy: 0.1600 - val_loss: 2.3722 - val_accuracy: 0.1039


Once again, ScalarStop saves the model's trainining history alongside the model's weights, but this is not very convenient if you want to do large-scale analysis on the training metrics of many models at once.

A better way of storing the training metrics is to use the `TrainStore`.

---
### `TrainStore`: Save and query your machine learning metrics in a database

In [16]:
train_store = sp.TrainStore.from_filesystem(filename="train_store.sqlite3")
train_store

<sp.TrainStore sqlite:///train_store.sqlite3>

The `TrainStore` is also available as a Python context manager.

```python
with sp.TrainStore.from_filesystem(filename="train_store.sqlite3") as train_store:
    # use the TrainStore here

# here the TrainStore database connection is automatically closed for you.
```

We don't use it that way in this example because we want to use the TrainStore across multiple Jupyter notebook cells.

The `TrainStore` will automatically save your `DataBlob` and `ModelTemplate` name, group name, and hyperparameters to the database. And when you train a `Model`, the `TrainStore` will persist the model name and the epoch training metrics.

All of this happens automatically if you pass the `TrainStore` instance to `Model.fit()`.

In [17]:
_ = model.fit(final_epoch=5, train_store=train_store)

Epoch 3/5
25/25 [==============================] - 2s 90ms/step - loss: 2.2991 - accuracy: 0.1600 - val_loss: 2.3707 - val_accuracy: 0.1039
Epoch 4/5
25/25 [==============================] - 2s 89ms/step - loss: 2.2978 - accuracy: 0.1600 - val_loss: 2.3705 - val_accuracy: 0.1039
Epoch 5/5
25/25 [==============================] - 2s 87ms/step - loss: 2.2965 - accuracy: 0.1600 - val_loss: 2.3705 - val_accuracy: 0.1039


Once you have some information in the `TrainStore`, you can query it for information and receive results as a Pandas `DataFrame`.

In [18]:
train_store.list_datablobs()

,name,group_name,hyperparams,last_modified
0,fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza,fashion_mnist_v1,{'num_training_samples': 50},2021-04-08 04:13:16.040227


In [19]:
train_store.list_model_templates()

,name,group_name,hyperparams,last_modified
0,small_dense_10_way_classifier_v1-uptyfbjofo7rq...,small_dense_10_way_classifier_v1,"{'hidden_units': 3, 'optimizer': 'adam'}",2021-04-08 04:13:16.041786


In [20]:
train_store.list_models()

,model_name,model_class_name,model_last_modified,datablob_name,datablob_group_name,model_template_name,model_template_group_name,dbh__num_training_samples,mth__hidden_units,mth__optimizer
0,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,KerasModel,2021-04-08 04:13:16.042950,fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza,fashion_mnist_v1,small_dense_10_way_classifier_v1-uptyfbjofo7rq...,small_dense_10_way_classifier_v1,50,3,adam


In [21]:
train_store.list_model_epochs(model_name=model.name)

,epoch_num,model_name,last_modified,metric__loss,metric__accuracy,metric__val_loss,metric__val_accuracy
0,3,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-08 04:13:18.306168,2.299144,0.16,2.370655,0.1039
1,4,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-08 04:13:20.526185,2.297823,0.16,2.370520,0.1039
2,5,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-08 04:13:22.697464,2.296533,0.16,2.370511,0.1039


In [22]:
model_template = small_dense_10_way_classifier_v1(hyperparams=dict(hidden_units=5))

In [23]:
model = sp.KerasModel.from_model_template(datablob=datablob, model_template=model_template)

In [24]:
_ = model.fit(final_epoch=10, train_store=train_store)

Epoch 1/10
25/25 [==============================] - 2s 90ms/step - loss: 21.2974 - accuracy: 0.1000 - val_loss: 2.5189 - val_accuracy: 0.1036
Epoch 2/10
25/25 [==============================] - 2s 87ms/step - loss: 2.3525 - accuracy: 0.1600 - val_loss: 2.3789 - val_accuracy: 0.1035
Epoch 3/10
25/25 [==============================] - 2s 91ms/step - loss: 2.2993 - accuracy: 0.1600 - val_loss: 2.3727 - val_accuracy: 0.1035
Epoch 4/10
25/25 [==============================] - 2s 93ms/step - loss: 2.2981 - accuracy: 0.1600 - val_loss: 2.3722 - val_accuracy: 0.1035
Epoch 5/10
25/25 [==============================] - 2s 93ms/step - loss: 2.2969 - accuracy: 0.1600 - val_loss: 2.3722 - val_accuracy: 0.1035
Epoch 6/10
25/25 [==============================] - 2s 93ms/step - loss: 2.2957 - accuracy: 0.1600 - val_loss: 2.3722 - val_accuracy: 0.1035
Epoch 7/10
25/25 [==============================] - 2s 89ms/step - loss: 2.2945 - accuracy: 0.1600 - val_loss: 2.3722 - val_accuracy: 0.1035
Epoch 8/10
2

In [25]:
train_store.list_model_epochs(model_name=model.name)

,epoch_num,model_name,last_modified,metric__loss,metric__accuracy,metric__val_loss,metric__val_accuracy
0,1,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:25.232950,21.297380,0.10,2.518939,0.1036
1,2,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:27.409848,2.352500,0.16,2.378875,0.1035
2,3,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:29.692196,2.299279,0.16,2.372684,0.1035
3,4,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:32.020865,2.298073,0.16,2.372222,0.1035
4,5,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:34.339693,2.296874,0.16,2.372190,0.1035
5,6,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:36.657438,2.295686,0.16,2.372182,0.1035
6,7,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:38.895995,2.294514,0.16,2.372215,0.1035
7,8,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:41.118748,2.293363,0.16,2.372278,0.1035
8,9,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:43.369257,2.292232,0.16,2.372333,0.1035
9,10,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-08 04:13:45.599492,2.291124,0.16,2.372398,0.1035
